In [1]:
import OpenDartReader
import pandas as pd 

df = pd.read_csv('esg_finance_data.csv', dtype={'corp_code': 'str', 'stock_code': 'str'})
df['year'] = ['2020'] * 847 + ['2021'] * 847 + ['2022'] * 847
df['year'] = df['year'].astype('int')
print(df.dtypes)
print(df.shape, len(df.corp_code.unique())) #847
df.head(3)

corp_name     object
esg           object
fin_state     object
corp_code     object
stock_code    object
year           int32
dtype: object
(2541, 6) 847


,corp_name,esg,fin_state,corp_code,stock_code,year
0,기아,A+,"60,490,443,000,000",00106641,000270,2020
1,S-Oil,A+,"15,690,510,000,000",00138279,010950,2020
2,풀무원,A+,"1,693,573,176,270",00155355,017810,2020


In [3]:
# 매출액 데이터
# api_key ='4fb4acc1d37b1dae74040e64a9674e018ccfd7fe'
api_key = '1ee244d6168bde153cb2d45463d71139f567d1f4'
dart = OpenDartReader(api_key)

def sales_docu(code, year):
    sales = dart.finstate(code, year)
    
    if sales.shape[1] >= 8:
        sale_df = sales.iloc[:, 8:]
    else:
        sale_df = sales
    # real_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')].iloc[0, :] # 매출액
    # c_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')]['thstrm_amount'].values # 매출액 # 문제1
    # p_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')]['frmtrm_amount'].values # t-1기 매출액 # 문제 2

    div_cash = dart.report(code, '배당', year) #(연결)현금배당성향(%)
    if div_cash.shape[1] >= 3:
        cash_df = div_cash.iloc[:, 3:]
    else:
        cash_df = div_cash
    # c_div = div_cash[div_cash['se']=='(연결)현금배당성향(%)']['thstrm'].values # 순이익이 마이너스일 경우 0% # 배당금 / 당기순이익
    # p_div = div_cash[div_cash['se']=='(연결)현금배당성향(%)']['frmtrm'].values

    holder = dart.report(code, '최대주주', year) # 보통주 계 + 우선수 계 (%)
    # ratio_stakeholder = holder[(holder['stock_knd']=='보통주')&(holder['nm']=='계')].values # 문제3

    exec = dart.report(code, '임원', year)
    if exec.shape[1] >= 3:
        exec_df = exec.iloc[:, 3:]
    else:
        exec_df = exec
    #'사외이사' / 전체 임원 수   
    # out_exec = exec[exec['ofcps'] == '사외이사'].shape[0]
    # exec = exec.shape[0]

    employee = dart.report(code, '직원', year) # 총 직원 수
    
    # output = {'sales': c_sales, 
    #           'pre_sales': p_sales, 
    #           'cash_div': c_div, 
    #           'pre_cash_civ': p_div,
    #           'stakeholder' : ratio_stakeholder,
    #           'num_outexecutives': out_exec,
    #           'num_executives': exec,
    #           'num_employee': employee}
    
    return sale_df, cash_df, exec_df, employee

In [5]:
# sales_docu('00106641', 2022)

# 2022 매출 데이터

In [6]:
good_index, bad_index = [], []

for index, code in enumerate(df['corp_code'].unique()): 
    try:
        sales, cash, exec, emp = sales_docu(code, 2022)
        sales.to_csv(f'./2022/sales_2022/{code}_{str(2022)}.csv', encoding='utf8')
        cash.to_csv(f'./2022/cash_2022/{code}_{str(2022)}.csv', encoding='utf8')
        exec.to_csv(f'./2022/exec_2022/{code}_{str(2022)}.csv', encoding='utf8')
        emp.to_csv(f'./2022/emp_2022/{code}_{str(2022)}.csv', encoding='utf8')
        print(f'{index}, Good!') # check if it works well
        good_index.append(index)
    except:
        print(f'{index}, bad...') # check if there is a problem
        bad_index.append(index)
        
print(len(good_index), len(bad_index))

0, Good!
1, Good!
2, Good!
3, Good!
4, Good!
5, Good!
6, Good!
7, Good!
8, Good!
9, Good!
10, Good!
11, Good!
12, Good!
13, Good!
14, Good!
15, Good!
16, Good!
17, Good!
18, Good!
19, Good!
20, Good!
21, Good!
22, Good!
23, Good!
24, Good!
25, Good!
26, Good!
27, Good!
28, Good!
29, Good!
30, Good!
31, Good!
32, Good!
33, Good!
34, Good!
35, Good!
36, Good!
37, Good!
38, Good!
39, Good!
40, Good!
41, Good!
42, Good!
43, Good!
44, Good!
45, Good!
46, Good!
47, Good!
48, Good!
49, Good!
50, Good!
51, Good!
52, Good!
53, Good!
54, Good!
55, Good!
56, Good!
57, Good!
58, Good!
59, Good!
60, Good!
61, bad...
62, Good!
63, Good!
64, Good!
65, Good!
66, Good!
67, Good!
68, Good!
69, Good!
70, Good!
71, Good!
72, Good!
73, Good!
74, Good!
75, Good!
76, Good!
77, Good!
78, Good!
79, Good!
80, Good!
81, Good!
82, Good!
83, Good!
84, Good!
85, Good!
86, Good!
87, Good!
88, Good!
89, Good!
90, Good!
91, Good!
92, Good!
93, Good!
94, Good!
95, Good!
96, Good!
97, Good!
98, Good!
99, Good!
100, Good

In [ ]:
# good_index, bad_index = [], []

# for index, code in enumerate(df['stock_code'].unique()): 
#     try:
#         fin, inc = sales_docu(code, 2020)
#         fin.to_csv(f'./2020/fin_2020/{code}_{str(2020)}.csv', encoding='utf8')
#         inc.to_csv(f'./2020/inc_2020/{code}_{str(2020)}.csv', encoding='utf8')
          
#         print(f'{index}, Good!') # check if it works well
#         good_index.append(index)
#     except:
#         print(f'{index}, bad...') # check if there is a problem
#         bad_index.append(index)
        
# print(len(good_index), len(bad_index))

# 2020 매출 데이터

In [7]:
good_index, bad_index = [], []

for index, code in enumerate(df['corp_code'].unique()): 
    try:
        sales, cash, exec, emp = sales_docu(code, 2020)
        sales.to_csv(f'./2020/sales_2020/{code}_{str(2020)}.csv', encoding='utf8')
        cash.to_csv(f'./2020/cash_2020/{code}_{str(2020)}.csv', encoding='utf8')
        exec.to_csv(f'./2020/exec_2020/{code}_{str(2020)}.csv', encoding='utf8')
        emp.to_csv(f'./2020/emp_2020/{code}_{str(2020)}.csv', encoding='utf8')
        print(f'{index}, Good!') # check if it works well
        good_index.append(index)
    except:
        print(f'{index}, bad...') # check if there is a problem
        bad_index.append(index)
        
print(len(good_index), len(bad_index))

0, Good!
1, Good!
2, Good!
3, Good!
4, Good!
5, Good!
6, Good!
7, Good!
8, Good!
9, Good!
10, Good!
11, Good!
12, Good!
13, Good!
14, Good!
15, Good!
16, Good!
17, Good!
18, Good!
19, Good!
20, Good!
21, Good!
22, Good!
23, Good!
24, Good!
25, Good!
26, Good!
27, Good!
28, Good!
29, Good!
30, Good!
31, Good!
32, Good!
33, Good!
34, Good!
35, Good!
36, Good!
37, Good!
38, Good!
39, Good!
40, Good!
41, Good!
42, Good!
43, Good!
44, Good!
45, Good!
46, Good!
47, Good!
48, Good!
49, Good!
50, Good!
51, Good!
52, Good!
53, Good!
54, Good!
55, Good!
56, Good!
57, Good!
58, Good!
59, Good!
60, Good!
61, Good!
62, Good!
63, Good!
64, Good!
65, Good!
66, Good!
67, Good!
68, Good!
69, Good!
70, Good!
71, Good!
72, Good!
73, Good!
74, Good!
75, Good!
76, Good!
77, Good!
78, Good!
79, Good!
80, Good!
81, Good!
82, Good!
83, Good!
84, Good!
85, Good!
86, Good!
87, Good!
88, Good!
89, Good!
90, Good!
91, Good!
92, Good!
93, Good!
94, Good!
95, Good!
96, Good!
97, Good!
98, Good!
99, Good!
100, Good!

In [ ]:
# # importng data
# data, good_index, bad_index = [], [], []

# for index, (code , year) in enumerate(zip(df['stock_code'], df['year'])):
#     try:
#         data.append((code, year, sales_docu(code, year)))
#         good_index.append(index)
#     except:
#         bad_index.append(index)
# print(len(good_index), len(bad_index))